In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import networkx as nx
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
fp1 = "/Users"
df_01 = pd.read_csv(fp1, low_memory = False, encoding = 'utf-8')  #get the Jan-2020 spreadsheet
df_01 = df_01[df_01.author != '[deleted]'] #remove authors that are deleted

In [ ]:
# create a function that works out the timestamps of each day in a month
def workout_time(year, month):
  max_day = 31
  month_list = [4, 6, 9, 11]

  if year == 2020 and month == 2:
    max_day = 29
  elif month == 2:
    max_day = 28
  elif month in month_list:
    max_day = 30
  
  start_date = datetime(year, month, 1, 0) # always start at the first day on each month

  day_list = []

  for i in range(max_day):
    count_time = start_date + relativedelta(days=i)
    count_time = count_time.timestamp()
    day_list.append(count_time)
  
  return day_list

# a funtion that creates sub_dataframes from the parent dataframe based on number of days
def create_subframe(day_list, parent_dataframe):
  df_list = []

  for i in range(len(day_list)-1):
    left = day_list[i]
    right = day_list[i+1]

    day_df = parent_dataframe[(parent_dataframe.created_utc >= left) & (parent_dataframe.created_utc < right)]
    
    df_list.append(day_df)
  
  return df_list

# a funtion that creates sub_dataframes from the parent dataframe based on number of days
def create_subframe(day_list, parent_dataframe):
  df_list = []

  for i in range(len(day_list)-1):
    left = day_list[i]
    right = day_list[i+1]

    day_df = parent_dataframe[(parent_dataframe.created_utc >= left) & (parent_dataframe.created_utc < right)]
    
    df_list.append(day_df)
  
  return df_list

In [ ]:
year, month = 2020, 1
day_list_01_20 = workout_time(year, month)
df_list_01_20 = create_subframe(day_list_01_20, df_01)   #creates day-by-day sub-dataframes

In [ ]:
#bot removal
def AinB (A, B):
    check =  any(item in A for item in B)
    return check

def del_bot(name):
    tokens = re.split('[^a-zA-Z]', name)
    
    not_list = ['not', 'Not', 'NOt', 'NOT', 'nOt', 'nOT', 'noT', 'NoT']
    
    bot_list = ['bot', 'Bot', 'BOt', 'BOT', 'bOt', 'bOT', 'boT', 'BoT']
    
    if AinB(not_list, tokens) and AinB(bot_list, tokens):
        return False

    elif AinB(bot_list, tokens):
        return True
    
    return False

In [ ]:
drop_indices = []


    drop_indices
    
def create_parent_children_df(dataframe):     #create the output dataframe
    tqdm.pandas()
    level_1_comments = dataframe[dataframe['parent_id'] == dataframe['link_id']]

    sub_comments = dataframe[~dataframe.index.isin(level_1_comments.index)] #setting the sub comments being those with index that are not in level_1_comments

    posts_authors  = sub_comments.groupby('parent_id')['author'].apply(list).reset_index(name='author')
    posts_authors['sub_comments'] = posts_authors['author'].progress_apply(lambda x:len(x))
    posts_authors.sort_values('sub_comments', inplace=True, ascending=False)

    return posts_authors

def create_name_id_dict(dataframe):    #create id:author dataframe
    name_id = {}
    for i in range(dataframe.shape[0]):
        key = dataframe.id.iloc[i]
        value = dataframe.author.iloc[i]

        key_value_pair = {key:value}

        name_id.update(key_value_pair)
    return name_id

def clean_by_parent(parent_children_dataframe, name_id_dicitonary):   #clean bots from parent_id
    drop_indices = []
    parent_ids = parent_children_dataframe.parent_id
    indices = parent_children_dataframe.index
    for i in range(len(parent_ids)):
        id = parent_ids[i].split('_')[-1]
        # use the dictionary to find the corresponding author name
        try:
            author_name = name_id_dicitonary[id]
        except:
            continue
        if del_bot(author_name):
            drop_indices.append(indices[i])
    
    return drop_indices

def clean_author(parent_children_dataframe):    #clean bots from authors
    for i in range(parent_children_dataframe.shape[0]):
        bot = False
        df_index = parent_children_dataframe.index
        author_list = parent_children_dataframe.author.iloc[i]
        # print(author_list)
        for author in author_list:
            if del_bot(author):
                author_list.remove(author)
                bot = True
                # print(1)
        if bot:
            # print(1)
            parent_children_dataframe.loc[df_index[i]].at['author'] = author_list
            parent_children_dataframe.loc[df_index[i]].at['sub_comments'] = len(author_list)
    
    return parent_children_dataframe

In [ ]:
day_df = df_list_01_20[1]
posts_authors = create_parent_children_df(day_df)
name_id = create_name_id_dict(day_df)
drop_indices = clean_by_parent(posts_authors, name_id)
drop_indices
posts_authors = posts_authors.drop(drop_indices)
posts_authors = clean_author(posts_authors)
posts_authors

In [ ]:
def get_all_authors(name_id_dict, dataframe):   #get all author names
    author_list = set()
    for i in range(dataframe.shape[0]):
        uid = dataframe.parent_id.iloc[i].split('_')[-1]

        try:
            author_list.add(name_id_dict[uid])
        except:
            continue

        author_list.update(tuple(dataframe.author.iloc[i]))
        
    return author_list

def generate_adj_matrix(author_list, dataframe, name_id_dict):   #generate the adjacency matrix
    adj_matrix = pd.DataFrame(0, index = author_list, columns = author_list)
    for i in range(dataframe.shape[0]):
        uid = dataframe.parent_id.iloc[i].split('_')[-1]
        
        try:
            parent_name = name_id_dict[uid]
        except:
            continue
        author_list = dataframe.author.iloc[i]

        for author in author_list:
            adj_matrix.loc[author, parent_name] += 1
    
    return adj_matrix

In [ ]:
author_list = get_all_authors(name_id, posts_authors)
generate_adj_matrix(author_list, posts_authors, name_id)
adj_matrix.to_csv('file name')